# TKO_7092 Evaluation of Machine Learning Methods 2024

---

Student name: Faiza Anan Noor

Student number: 2306676

Student email: fanoor@utu.fi


---

## Exercise 4

Complete the tasks given to you in the letter below. In your submission, explain clearly, precisely, and comprehensively why the cross-validation described in the letter failed, how cross-validation should be performed in the given scenario and why  your cross-validation will give a reliable estimate of the generalisation performance. Then implement the correct cross-validation for the scenario and report its results.

Remember to follow all the general exercise guidelines that are stated in Moodle. Full points (2p) will be given for a submission that demonstrates a deep understanding of cross-validation on pair-input data and implements the requested cross-validation correctly (incl. reporting the results). Partial points (1p) will be given if there are small error(s) but the overall approach is correct. No points will be given if there are significant error(s).

The deadline of this exercise is **Wednesday 21 February 2024 at 11:59 PM**. Please contact Juho Heimonen (juaheim@utu.fi) if you have any questions about this exercise.

---


Dear Data Scientist,

I have a long-term research project regarding a specific set of proteins. Currently I am attempting to discover small organic compounds that can bind strongly to these proteins and thus act as drugs. I have a list of over 100.000 potential drug molecules, but their affinities still need to be verified in the lab. Obviously I do not have the resources to measure all the possible drug-target pairs, so I need to prioritise. I have decided to do this with the use of machine learning, but I have encountered a problem.

Here is what I have done so far: First I trained a K-nearest neighbours regressor with the parameter value K=10 using all the 400 measurements I had made in the lab, which comprise of all the 77 target proteins of interest but only 59 different drug molecules. Then I performed a leave-one-out cross-validation with this same data to estimate the generalisation performance of the model. I used C-index and got a stellar score above 90%. Finally I used the model to predict the affinities of the remaining drug molecules. The problem is: when I selected the highest predicted affinities and tried to verify them in the lab, I found that many of them are much lower in reality. My model clearly does not work despite the high cross-validation score.

Please explain why my estimation failed and how leave-one-out cross-validation should be performed to get a reliable estimate. Also, implement the correct leave-one-out cross-validation and report its results. I need to know whether I am wasting my lab resources by using my model.

The data I used to create my model is available in the files `input.data`, `output.data` and `pairs.data` for you to use. The first file contains the features of the pairs, whereas the second contains their affinities. The third file contains the identifiers of the drug and target molecules of which the pairs are composed. The files are paired, i.e. the i<sup>*th*</sup> row in each file is about the same pair.

Looking forward to hearing from you soon.

Yours sincerely, \
Bio Scientist

---

#### Answer the questions about cross-validation on pair-input data

In [1]:
# Why did the estimation described in the letter fail?
# How should leave-one-out cross-validation be performed in the given scenario and why?
# Remember to provide comprehensive and precise arguments.

###  Why did the estimation described in the letter fail?
The estimation failed because the leave out one cross validation method implementation was faulty as the bioscientist plainly used leave one out not considering the fact  that the drug-input pairs was in scenario B. The letter also says that all the 400 measurements the bioscientist had made in the lab comprise of all the 77 target proteins of interest but only 59 different drug molecules. It is also notable  that  our pairs.data file shows that  the first column that is the drug molecule is out of sample sometimes whwereas all the target molecules are in sample. This falls within the setting B setting. In the case B, the first pair members of out-of-sample observations are out-of-sample objects. Hence the first pair members of the test observations are not allowed in the training set. The training set contains  all in-sample observations  except those that share their first pair members with any of the test observations. Test observations are always excluded by this condition. 

But the choice of leave one out considered the setting to be in setting A where it considers all pair members to be in sample which is not the case for us. A hypothesis is expected to perform best on type A observations and worst on type D observations, with type B and C observations in between. This is because the dependencies between an out-of-sample observation and the sample diminish as the number of objects they share decreases. And this is why the bioscientist got unrealistically optimistic results and got the stellar score of 90% as he considered both pair members to be in sample but in practicality they are not. So when they are implemented in the appropriate setting(setting B), a more realistic estimation can be obtained which depicts the actual performance of the model in this setting. 



###  How should leave-one-out cross-validation be performed in the given scenario and why?

In cases where there is a significant class imbalance between the number of targets and the number of samples (in this case, 77 targets but only 59 drug molecules), Leave Drug Out (LDO) cross-validation may be the most suitable choice. Therefore, for cases with a small number of drug molecules compared to the number of targets, it is advisable to use cross-validation techniques that account for the class imbalance and provide more reliable estimates of model performance. LOOCV may result in the model memorizing specific characteristics of individual molecules rather than generalizing well to new, unseen molecules. This can lead to inflated performance estimates during cross-validation, as observed with the reported stellar score above 90%.

To adapt the estimation of out-of-sample performance to pair-input data, one only needs to modify the selection of training observations. In the selection of training observations, the general principle is that  the set of objects that appear as pair members of training observations must be restricted. The restriction controls the dependencies between the test and training sets.  An out-of-sample observation of type B( setting B)  has only its second pair member shared with the sample.  Its dependencies on the sample are weaker than those of the type A observations. The regular leave-one-out cross-validation must hence be modified to match those dependencies in order to avoid optimistically biased estimates. 


To summarise,  performance evaluation methods can be adapted to each of the four types of out-of-sample pair-input observations. The key is to not use those in-sample observations for training that violate the condition determined by the out-of-sample observation type.  This replicates the dependencies between the out-of-sample observations and the sample. This ensures that the model learns to generalize patterns from a diverse set of molecules, leading to more accurate predictions on unseen molecules. For our case also, we try to simulate this scenario(setting B) by modifying the plain Leave Out One(LOO) Cross validation such that for each iteration when we choose the i-th test set, we discard all drugs of that particular iteration and use the rest of the indices to extract from train labels and features. This is because the test observation  must not share its first pair member with any of the training observations for this setting. To fulfill this condition,  the in-sample observations that share their first pair members with the test observation must not be used for training.  Those observations are simply ignored. 


#### Import libraries

In [2]:
# Import the libraries you need.

import numpy as np
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
from sklearn.preprocessing import StandardScaler

#### Write utility functions

In [3]:
# Write the utility functions you need in your analysis.
# C-index score from previous exercises
def cindex(true_labels, pred_labels):
    n = 0
    h_num = 0 
    for i in range(0, len(true_labels)):
        t = true_labels[i]
        p = pred_labels[i]
        for j in range(i+1, len(true_labels)):
            nt = true_labels[j]
            np = pred_labels[j]
            if (t != nt): 
                n = n + 1
                if (p < np and t < nt) or (p > np and t > nt): 
                    h_num += 1
                elif (p == np):
                    h_num += 0.5 
    cindx = h_num  /n
    return cindx

#### Load datasets

In [4]:
# Read the data files (input.data, output.data, pairs.data).
# Load data

X = pd.read_csv('input.data', header=None, sep = " ").values
y = pd.read_csv('output.data', header=None, sep = " ").values
pairs = pd.read_csv('pairs.data',header=None, sep = " ").values

In this cell, we standardize the dataset features by removing the mean and scaling to unit variance. In other words, we used z-score to scale the dataset features so that it ensures all features are equally penalized, ultimately enhancing model performance, stability, and interpretability.


In [5]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to your data and transform it
X_scaled = pd.DataFrame(scaler.fit_transform(X)).values

In [ ]:
X_scaled

#### Implement and run cross-validation

In [ ]:
preds2 = []

model = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
drugs=pairs[:, 0]
for i in range(len(X_scaled)):
    # Select the ith number row as the test set and reshpe as necessary

    X_test = X_scaled[i].reshape(1, -1)

    # Save the ith target for train set selection
    drug = drugs[i]
    # Choose train target and label from rows that doesn't contain the drug for that particular iteration for ith row
    y_train = y[(drug != drugs)]
    X_train = X_scaled[(drug != drugs)]
    #Fitting the model    
    model.fit(X_train, y_train)
    # Prediction on test set
    prediction=(model.predict(X_test))
    #Appending predictions to pred2 array to use to calcullate cindex
    preds2.append(prediction)
    
# Calculating Cindex    
c_index=cindex(y, preds2)
print("C-index score predicting affinities of targets to drug molecules: ",c_index)

We can see that the experiment run in setting B( first pair member out of sample, second in sample) yielded a much less result that is 51.3% compared to 90% as before but this is a more realistic estimation as previously it was producing overly optimal results considering both pair members to be in sample. Like I mentioned before, typically both in pair members produce much more significantly better results however that wasnt actually the practical setting for our scenario(scenario A). This is why when the bioscientist used the model to predict the affinities of the remaining drug molecules tried to verify them in the lab, he found that many of them are much lower in reality. His model clearly does not work despite the high cross-validation score.